This Python notebook is for the rainfall python exercises.

Load library for code formatter

In [1]:
%load_ext lab_black

Load dependencies

In [2]:
import pandas as pd
import math
import numpy as np
import datetime

List down constants

In [3]:
excel_path = "../data/eo-exercise/PCC Daily Data 2006-2017.xlsx"
csv_path = "../data/eo-exercise/locations.csv"
update_data_path = "../data/eo-exercise/201910073 - RainData from Pagasa"
headerrows = 15
footerrows = 7

In [4]:
sheet_to_df = pd.read_excel(
    excel_path,
    sheet_name=None,
    skiprows=headerrows,
    skipfooter=footerrows,
    usecols="A:H",
)

In [5]:
sheet_to_df["NAIA"]

,YEAR,MONTH,DAY,RAINFALL,TMEAN,RH,WIND SPEED,WIND DIRECTION
0,2006.0,1,1,0,28.0,85.5,2,110
1,2006.0,1,2,-2,26.7,87.5,2,110
2,2006.0,1,3,0,27.5,75.8,2,110
3,2006.0,1,4,0,27.3,76.2,2,110
4,2006.0,1,5,0,26.4,80.8,2,120
...,...,...,...,...,...,...,...,...
4378,2017.0,12,27,1.5,26.2,100.0,2,70
4379,2017.0,12,28,0,23.3,77.0,3,70
4380,2017.0,12,29,0,22.0,76.0,3,70
4381,2017.0,12,30,0,21.1,74.0,3,70


In [6]:
sheet_to_df["NAIA"].iloc[0, :]

YEAR              2006
MONTH                1
DAY                  1
RAINFALL             0
TMEAN               28
RH                85.5
WIND SPEED           2
WIND DIRECTION     110
Name: 0, dtype: object

In [7]:
sheet_to_df["NAIA"].iloc[:, 0]

0       2006.0
1       2006.0
2       2006.0
3       2006.0
4       2006.0
         ...  
4378    2017.0
4379    2017.0
4380    2017.0
4381    2017.0
4382    2017.0
Name: YEAR, Length: 4383, dtype: float64

In [8]:
sheet_to_df["NAIA"].iloc[9, 6]

2

In [9]:
sheet_names = sheet_to_df.keys()

single_df = pd.concat(sheet_to_df, keys=sheet_names)

In [10]:
field_names = single_df.keys()

df_types = {
    "YEAR": int,
    "MONTH": int,
    "DAY": int,
    "RAINFALL": float,
    "TMEAN": float,
    "RH": int,
    "WIND SPEED": int,
    "WIND DIRECTION": int,
}

In [11]:
miss = {}


def parse_error_columns(field):
    field_type = df_types[field]

    for item in single_df[field]:
        try:
            if field_type == int:
                int(item)
            elif field_type == float:
                float(item)
            else:
                print(item)
        except:
            try:
                math.isnan(item)
            except:
                try:
                    miss[item] = miss[item] + 1
                except:
                    miss[item] = 1

In [12]:
for field in field_names:
    try:
        single_df[field].astype(df_types[field])
    except ValueError as e:
        parse_error_columns(field)

In [13]:
miss

{'T': 2409,
 't': 19,
 ' /': 1,
 '3.8/': 1,
 '1.1.2': 1,
 '6.8.': 1,
 'TT': 1,
 '/': 60,
 '24..6': 1,
 '27.8.': 1,
 '29.4.': 1,
 '24.6.': 1,
 'VAR': 43,
 'VRB': 1}

In [14]:
single_df = single_df.reset_index()
del single_df["level_1"]
single_df["locations"] = single_df.pop("level_0")

In [15]:
miss_fields = list(miss.keys())

single_df = single_df.replace(miss_fields, np.nan)

In [16]:
csv_to_df = pd.read_csv(csv_path)

merged_df = pd.merge(single_df, csv_to_df, on="locations", how="left")

In [17]:
def create_datetime(year, month, day):
    try:
        new_date = datetime.datetime(int(year), int(month), int(day))
        return new_date
    except:
        return np.nan

In [28]:
date_array = []

for i in range(len(merged_df["YEAR"])):
    row_date = create_datetime(
        merged_df["YEAR"][i], merged_df["MONTH"][i], merged_df["DAY"][i]
    )
    date_array.append(row_date)

In [19]:
def celsius_to_fahrenheit(cel):
    f = (float(cel) * (9 / 5)) + 32
    return f

In [20]:
merged_df["TMEAN"] = [celsius_to_fahrenheit(x) for x in merged_df["TMEAN"]]

In [21]:
location_list = set(merged_df["locations"])

update_data = {}

for location in location_list:
    try:
        data = pd.read_csv("%s/%s.csv" % (update_data_path, location))
        data["locations"] = [location for x in range(len(data["YEAR"]))]
        update_data[location] = data
    except:
        print("No file found for %s" % (location))

obj_names = update_data.keys()

single_update_df = pd.concat(update_data, keys=obj_names)
single_update_df = single_update_df.reset_index(drop=True)

No file found for Maasin
No file found for Catbalogan
No file found for Lumbia
No file found for El Salvador
No file found for Cotabato
No file found for Guiuan
No file found for Mactan
No file found for Borongan
No file found for General Santos
No file found for Catarman
No file found for Butuan
No file found for Roxas City
No file found for Surigao
No file found for Hinatuan
No file found for Zamboanga
No file found for Malaybalay
No file found for Davao City
No file found for Tacloban
No file found for Dipolog
No file found for Dumaguete


In [26]:
merged_df["NEW_INDEX"] = merged_df.apply(
    lambda row: "%d - %d - %d - %s"
    % (row["DAY"], row["MONTH"], row["YEAR"], row["locations"]),
    axis=1,
)

single_update_df["NEW_INDEX"] = single_update_df.apply(
    lambda row: "%d - %d - %d - %s"
    % (row["DAY"], row["MONTH"], row["YEAR"], row["locations"]),
    axis=1,
)

merged_df = merged_df.set_index("NEW_INDEX")
single_update_df = single_update_df.set_index("NEW_INDEX")

In [23]:
merged_df = merged_df.combine_first(single_update_df)
merged_df.update(single_update_df)
merged_df.reset_index(drop=True)

,DAY,MONTH,RAINFALL,RH,TMEAN,WIND DIRECTION,WIND SPEED,YEAR,elevation,lat,locations,lon
0,1.0,1.0,47.2,83.2,78.80,40.0,3,2006.0,5.00,14.100000,Alabat,122.016667
1,1.0,1.0,4.0,83.5,80.24,40.0,1,2006.0,11.00,14.083333,Ambulong,121.050000
2,1.0,1.0,0.0,91.8,77.54,40.0,5,2006.0,3.00,18.350000,Aparri,121.616667
3,1.0,1.0,0.0,89.5,67.10,140.0,2,2006.0,1510.08,16.400000,Baguio,120.600000
4,1.0,1.0,0.9,96.4,77.00,350.0,1,2006.0,173.00,15.733333,Baler Radar,121.616667
...,...,...,...,...,...,...,...,...,...,...,...,...
130724,9.0,9.0,110.0,NaN,23.85,NaN,NaN,2018.0,NaN,NaN,Basco Radar,NaN
130725,9.0,9.0,0.5,NaN,28.65,NaN,NaN,2018.0,NaN,NaN,Cabanatuan,NaN
130726,9.0,9.0,0.0,NaN,28.55,NaN,NaN,2018.0,NaN,NaN,NAIA,NaN
130727,9.0,9.0,2.1,NaN,29.00,NaN,NaN,2018.0,NaN,NaN,Port Area,NaN


In [24]:
import csv

f = open("../data/eo-exercise/201910073 - RainData from Pagasa/ReadMe.txt", "r")
raw_data = f.readlines()
raw_data = [x.rstrip("\n") for x in raw_data]

useful_data = raw_data[7:]


def strip_words(line):
    stopwords = ["Latitude:", "N", "Longitude:", "E", "Elevation:", "m", "NA"]
    querywords = line.split()

    resultwords = [word for word in querywords if word not in stopwords]
    result = " ".join(resultwords)
    return result


locations = []
for x in useful_data:
    title = x.split("Latitude:")[0]
    locations.append(title.strip())

useful_data = [strip_words(x) for x in useful_data]

txt_list = []

for i in range(len(locations)):
    parsed = useful_data[i].split(" ")
    measurements = parsed[-3:]

    obj_name = locations[i]

    if len(measurements) == 3:
        txt_obj = {
            "locations": obj_name,
            "lat": measurements[0],
            "lon": measurements[1],
            "elevation": measurements[2],
        }
    else:
        txt_obj = {
            "locations": obj_name,
            "lat": np.nan,
            "lon": np.nan,
            "elevation": np.nan,
        }

    txt_list.append(txt_obj)

txt_df = pd.json_normalize(txt_list)

In [25]:
txt_df

,locations,lat,lon,elevation
0,Baguio,16.403992,120.60147,1510.08
1,Baler Radar,15.748809,121.632028,173
2,Cabanatuan,15.470387,120.951143,32
3,Clark,15.1717,120.5616667,151.564
4,Dagupan,16.086784,120.352045,2
5,Iba,15.328408,119.965661,5.538
6,Port Area,14.58841,120.967866,15
7,Infanta,14.74663611,121.6490333,7
8,Science Garden,14.645072,121.044282,42
9,NAIA,14.5047,121.004751,21
